In [1]:
from ortools.sat.python import cp_model
import itertools
from calendar import monthrange

In [11]:
def window(seq, n=2):
    "Returns a sliding window (of width n) over data from the iterable"
    "   s -> (s0,s1,...s[n-1]), (s1,s2,...,sn), ...                   "
    it = iter(seq)
    result = tuple(itertools.islice(it, n))
    if len(result) == n:
        yield result
    for elem in it:
        result = result[1:] + (elem,)
        yield result

In [4]:
staff_list = [['Olivia', 0],
              ['Emma', 1], 
              ['Ava', 1],
              ['Charlotte', 1], 
              ['Sophia', 0],
              ['Amelia', 0],
              ['Isabella', 1],
              ['Mia', 1],
              ['Evelyn', 1],
              ['Harper', 0],
              ['Camila', 0],
              ['Gianna', 0],
              ['Abigail', 0], 
              ['Luna', 0],
              ['Ella', 0],
              ['Elizabeth', 0], 
              ['Sofia', 1],
              ['Emily', 1],
              ['Avery', 0],
              ['Mila', 1]]

staff_length = len(staff_list)
staff = range(staff_length)

shift_list = ['0700 - 1500',
          '0730 - 1530 (FT)',
          '0930 - 1730',
          '1200 - 2000',
          '1400 - 2200',
          '1530 - 2330 (FT)',
          '1600 - 2400',
          '1800 - 0200',
          '2000 - 0400',
          '2200 - 0400',
          '2359 - 0700',
          'On call (22:00)']

midnight_shifts = [6,7,8,9,10,11]

shift_length = len(shift_list)
shifts = range(shift_length)

planning_period = monthrange(2021, 5)
days_length = planning_period[1]
days = range(planning_period[1])

In [5]:
model = cp_model.CpModel()

In [6]:
#decision variables
#staff 'm' works shift 's' on day 'd'
x = {(m, d, s) : \
    model.NewBoolVar('s%id%is%i' % (m, d, s)) \
    for m in staff \
    for d in days \
    for s in shifts}

In [7]:
# All shifts should be taken by doctors
# No two doctors in the same shift on the same day
for d in days:
    for s in shifts:
      constraint = [x[(m,d,s)] \
                    for m in staff]
      model.Add(sum(constraint) == 1)

In [8]:
# All shifts should be taken by doctors
# No two doctors in the same shift on the same day
for d in days:
    for s in shifts:
      constraint = [x[(m,d,s)] \
                    for m in staff]
      model.Add(sum(constraint) == 1)

In [9]:
# No two shifts same day
for m in staff:
  for d in days:
    constraint = [x[(m,d,s)] \
                  for s in shifts]
    model.Add(sum(constraint) <= 1)

In [12]:
# 2 days off after last midnight (except on call shift). (Modified)
for m in staff:
  for wind in window(days, 3):
    day1 = [x[(m,wind[0],s + midnight_offset)] \
            for s in midnight_shifts]
    day2 = [x[(m,wind[1],s)].Not() \
            for s in shifts]
    day3 = [x[(m,wind[2],s)].Not() \
            for s in shifts[:-1]]

    model.Add(sum(day1 + day2 + day3) < 1 + len(day2) + len(day3))

In [13]:
#solve and print
# model.Maximize(0)

solver = cp_model.CpSolver()
solver.Solve(model)

for d in days:
    print('Day', d)
    for m in staff:
        for s in shifts:
            if solver.Value(x[(m, d, s)]) == 1:
              print(staff_list[m], 'works shift', shift_list[s])
    print()

Day 0
['Olivia', 0] works shift 0930 - 1730
['Emma', 1] works shift 0700 - 1500
['Ava', 1] works shift 1200 - 2000
['Charlotte', 1] works shift 1530 - 2330 (FT)
['Sophia', 0] works shift 1400 - 2200
['Isabella', 1] works shift On call (22:00)
['Mia', 1] works shift 1800 - 0200
['Gianna', 0] works shift 1600 - 2400
['Ella', 0] works shift 0730 - 1530 (FT)
['Sofia', 1] works shift 2200 - 0400
['Emily', 1] works shift 2359 - 0700
['Mila', 1] works shift 2000 - 0400

Day 1
['Olivia', 0] works shift 2200 - 0400
['Emma', 1] works shift 2000 - 0400
['Amelia', 0] works shift 0730 - 1530 (FT)
['Isabella', 1] works shift On call (22:00)
['Mia', 1] works shift 1400 - 2200
['Evelyn', 1] works shift 1200 - 2000
['Camila', 0] works shift 1800 - 0200
['Gianna', 0] works shift 0700 - 1500
['Luna', 0] works shift 1600 - 2400
['Sofia', 1] works shift 1530 - 2330 (FT)
['Avery', 0] works shift 0930 - 1730
['Mila', 1] works shift 2359 - 0700

Day 2
['Olivia', 0] works shift 1400 - 2200
['Emma', 1] works sh